# Machine Learning Primer
## [Future Labs AI Summit, Oct. 30, 10-10:45am](http://aisummit2017.futurelabs.engineering.nyu.edu)
[![](images/future_labs.png)](http://futurelabs.nyc/)
## [Ross Fadely, AI Lead](https://www.linkedin.com/in/rossfadely/), [Insight Data Science](http://insightdata.ai)
### ross [at] insightdata [dot] ai
[![](images/Insight_small.png)](http://insightdata.ai)

<br><br><br><br><br><br>

# What is (and isn't) Machine Learning
![](images/what_is_ML.png)

# Breaking down the Machine Learning Landscape
![](images/supervised_vs_unsupervised.png)

# Supervised Learning
![](images/supervised.png)

# Unsupervised Learning
![](images/clustering.png)
![](images/dim_redux.png)
![](images/density_est.png)

# Diving in - Predicting Telemarketing success

As a working example, we are going to build a machine learning model to predict the success of a telemarketing campaign for a (unnamed) bank.  The original dataset can be found [here](http://archive.ics.uci.edu/ml/datasets/Bank+Marketing), but for our purposes we will use the dataset found in `bank_marketing.csv` in the github repo.
<br><br><br><br><br>

## Loading data and preprocessing

To start we are going to first load in the modules we are going to use.  Our dependencies here are numpy, pandas, and scikit-learn, all of which can be install via pip or using a environment package like Anaconda.

In [ ]:
# loading needed modules upfront.
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

Lets load our data into Pandas and have a look:

In [ ]:
data = pd.read_csv('./resources/bank_marketing.csv')
data.head()

Seems like we have 6 columns (or "features") in our data, followed by the thing we want to model - the "outcome".

In [ ]:
data.describe()

Almost all ML models require the data to be preprocessed.  Common preprocessing steps might include in-filling/removing missing data points, mean subtraction and unit-variance scaling (continuous data), and tokenization (NLP). 

In our example we can see all the data consists of categorical data.  In other words, all the entries for `job` fall into buckets like `housemaid`, `services`, or `admin`.  Note there is no easy way to represent these categories in numbers (e.g., what does it mean to have `housemaid` be a 1 and `services` to be a 2?).  To account for this most ML models require us to split each of these categories into their own features indicating if it is or isn't true.  This is called one-hot encoding or creating "dummy" variables.

In [ ]:
data = pd.get_dummies(data, columns =['job', 'marital', 'default', 'housing', 'loan', 'prev_outcome'])
data.head()

### Creating a training, validation, and test dataset

The whole point of creating a ML model is to try to develop a model that ***generalizes*** to ***new*** unseen data.  If a model works well for data it has in hand but fails when new users/examples are entered into the system, it is useless.

To assess generalization, we spit the data into special buckets we call training, test, and validation.
![](images/train_test_valid.png)
Our training data is what we use to have our ML model *learn*.  Later, to ultimately measure generalization we run *inference* (predict) on the test data.  Finally we have a third set of data we call validation data.  For better or worse, almost all ML models have parameters that drive how they work.  We use this special validation set to tune these, by using the validation data as "test" data during our search for the best hyperparameters.

**NOTE - ** Below we will use a simple `train_test_split` function to save 60% of our data for training, 20% for validation, and 20% for split.  Scikit-learn offers ways to do this in one shot as well as more robust (and almost always used) [cross validation approaches](http://scikit-learn.org/0.16/modules/classes.html#module-sklearn.cross_validation).  Also note, above we mention additional pre-processing techniques beyond one-hot encoding.  Often, these must be performed after splitting the data as to avoid a form of [data leakage](https://machinelearningmastery.com/data-leakage-machine-learning/).

In [ ]:
# break out our data and labels
X = data.iloc[:,1:]
y = data.iloc[:,0]

# split off 20% for test data
X_remaining, X_test, y_remaining, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# split off 20% for validation data
X_train, X_valid, y_train, y_valid = train_test_split(X_remaining, y_remaining, test_size=0.25, random_state=0)

print("We have {}, {}, {} train, validation, and test samples".format(y_train.size, y_valid.size, y_test.size))

# Fitting an initial model - logistic regression

As an initial approach, we are going to explore our predictive task using [Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) ([scikit's documentaion](http://scikit-learn.org/0.16/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)).  Logistic Regression is a relatively simple but incredibly useful model and with creative [feature engineering](https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/) can often rivel the performance of highly complicated and expensive to compute models.  Lets look at the math behind the model:

![](images/logistic2.png)

If we look inside the parenthesis, we see a simple linear model (remember y=mx+b?).  The exponential produces values from ~0 to infinity, which means the logistic function ranges from 0 to 1 - exactly what we want for binary classification.  

So how does one fit this to data?  First we need to define a cost or **loss function** which helps us define how well the current parameters (w and b here) fit the data.  For Logistic Regression this looks like:

![](images/logistic_loss2.png)

This may be a lot to unpack, but if you walk through the math, you will see this loss function is high when there is a mismatch between our predictions *f* and labels *y*.  To find the best models our code must optimize the parameters *w* and *b* to find the lowest value of the loss. Discussion of optimization is out of scope here, but know it often relies on variants of [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent).

<a id='Ex1'></a>
<div class="alert alert-info">
<h2>Exercise: Building intuition</h2><br>
Above we've shown you the model and cost function equations. To build some intuition about what those equations are doing, try plotting them out, i.e. plot `f(x)`, and `J(f,y)`. From these graphs, what can you say (in words) about their function in the model?
</div>

### Fitting LR in scikit-learn
Math aside, the process of fitting most ML models in scikit-learn is really straightforward:

In [ ]:
# call the model class
classifier= LogisticRegression(random_state=0)

# fit to the training data
classifier.fit(X_train, y_train)

# generate predictions for our test data!
y_pred = classifier.predict(X_valid)

Let's see how we did by looking at the *accuracy* - i.e., the percentage of examples we predicted correctly.  Scikit-learn includes this as a method in their LR class:

In [ ]:
print("We got {:0.2f} correct!".format(classifier.score(X_valid, y_valid)))

# Overfitting

Before we go further, let's take a crucial conceptual aside.  Of the core ideas that are key to successfully applying ML, overfitting is right at the top.  In short, overfitting is your model has learned the nuances and properties of your training data and performs significantly (sometimes *MUCH*) worse on your test data.  Consider the below example where a polynomial model is fit to data, using an increasingly higher degree ([borrowed from scikit-learn's website](http://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html#sphx-glr-auto-examples-model-selection-plot-underfitting-overfitting-py)):

![](images/overfitting.png)

When the degree is low, the model has little flexibility so the test error (here mean squared error or MSE) is reasonable but a little high.  As we increase the model's flexibility, we find a model with the right amount of wiggle to fit the training and test data well.  However if we continue to increase the capacity we will find we fit the training data *really* well but have a crazy function that can't possibly match the data in places the model hasn't seen.  Our goal when training ML models is to make sure this doesn't happen!

# Regularization

One of the main ways to combat overfitting is through regularization.  Simply put, regularization is the application of any approach that serves to limit model capacity in the hopes of getting better generalization performance. Typically, regularization comes in the form of penalizing parameter values from being too crazy or by cleverly restricting the structure of a model.  A complete review of regularization is also out of scope here, as it can vary quite significantly across different ML models.

Going back to our working example, we will briefly explore the two main forms of regularization for logistic regression namely L1 and L2 penalties.

## L2 Regularization

Probably the most common form of regularization across ML, L2 regularization penalizes model weight parameters for having large positive or negative values:

![](images/l2.png)

L2 is very straightforward to understand and compute, being just the sum of all the squared weight values.  This sum is added to our loss function, so that it is taken into account during training (optimization).  The lambda in front of the sum is called a hyperparamter (a parameter not learned during training but set upfront) and it controls how much the L2 penalty matters during training.

Looking at the scikit-learn API:

![](images/lr_sklearn.png)

We see that using L2 is set as default, and we have already been using it!  Lets explore the affect of varying the hyperparameter *C* (which is 1/lambda):

In [ ]:
def LR_model(X_train, y_train, X_valid, y_valid, lamb=1, rs=0, class_weight=None):
    model = LogisticRegression(C=1./lamb, random_state=rs, class_weight=class_weight)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    return y_pred, model.coef_, model.score(X_valid, y_valid), model

# fit lambda=1 as a reminder
lamb = 1
_, weights, acc, _ = LR_model(X_train, y_train, X_valid, y_valid, lamb)
print("We got {:0.2f} correct!".format(acc))

Great, lets look at the weights of the model to see what is happening under the good.

In [ ]:
print("lambda = {:2.2f}: We got {:0.2f} correct!".format(lamb, acc))
print("Here are the weights:\n", weights)
print("\nAnd the sum of their magnitudes is {:1.3f}".format(np.sum(np.abs(weights))))

Interesting, almost all of the weights carry some value with only one being zero.  What happens if we take C to large or small values?

In [ ]:
lamb = 1.e6
_, weights, acc, _ = LR_model(X_train, y_train, X_valid, y_valid, lamb=lamb)
print("lambda = {:2.2f}: We got {:0.2f} correct!".format(lamb, acc))
print("Here are the weights:\n", weights)
print("\nAnd the sum of their magnitudes is {:1.3f}".format(np.sum(np.abs(weights))))

lamb = 1.e-6
_, weights, acc, _ = LR_model(X_train, y_train, X_valid, y_valid, lamb=lamb)
print("\n\nlambda = {:2.2f}: We got {:0.2f} correct!".format(lamb, acc))
print("Here are the weights:", weights)
print("\nAnd the sum of their magnitudes is {:1.3f}".format(np.sum(np.abs(weights))))

# Class Imbalance

At this point you might be suspicious about the performance of our LR model - Why is the accuracy high?  Also, why does regularization only have a small effect?

It turns out that this dataset, like ***most*** datasets in the wild, suffers from a significant class imbalance.  That is, the number of negative examples (0's) greatly outnumber the positive examples (1's).  Lets look at this in detail:

In [ ]:
print("There are {:0.2f} negative examples in the data!".format(np.float(np.sum(y_test=='no')) / y_test.size))

In [ ]:
y_pred, weights, acc, _ = LR_model(X_train, y_train, X_valid, y_valid, lamb=1)
print("We predicted {:3d} positive samples in our validation data,".format(np.sum(y_pred=='yes')))
print("but there are {:3d} positive samples in our validation data!".format(np.sum(y_test=='yes')))

To better understand where our errors like, lets look at the confusion matrix - a grid that shows how many test examples our model says are positive/negative versus their true labels.

In [ ]:
%matplotlib inline
from resources.plot_utils import plot_confusion_matrix
fig = plot_confusion_matrix(confusion_matrix(y_valid, y_pred), ['negative','positive'])
fig.show()

### Addressing imbalance

Clearly the above results are an issue.  We are wrongly predicting that 744 people will not convert in the marketing campaign when in fact they will.  As a business, missing out the opportunity to contact high yield leads is not good.  One approach to address this is to directly tackle the class imbalance in the data.

[There are many approaches to addressing class imbalance](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/).  Perhaps the simplist and most straight forward is to reweight the examples in your loss function, according to their relative frequencies.  Luckily, most ML models in scikit-learn have this option baked in.  Below we will used this reweighting approach by setting `class_weight='balanced'`.

In [ ]:
y_pred, weights, acc, model = LR_model(X_train, y_train, X_valid, y_valid, lamb=1e-6, class_weight='balanced')
print("We got {:0.2f} correct!\n".format(acc))
fig = plot_confusion_matrix(confusion_matrix(y_valid, y_pred), ['negative','positive'])

Our total accuracy has gone down, but our ability to detect positives has dramatically improved. Determining what the [right metric](https://en.wikipedia.org/wiki/Precision_and_recall) for success depends a lot on the context of the problem.

<a id='Ex2'></a>
<div class="alert alert-info">
<h2>Exercise</h2><br>
Thus far, we've used the validation data to assess and tune the performance of our model. Of course, we can continue tuning to see if we can do better. But once we're done, we need to evaluate the performance of our model on the test set. This allows us to better understand how generalizable our model is. Your task is to calculate accuracy and re-generate the confusion matrix on the test set.
</div>

<a id='Ex2'></a>
<div class="alert alert-info">
<h2>Exercise</h2><br>
Try fitting a <a href=http://scikit-learn.org/0.16/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier>Random Forest</a> model to the data.  Does it do better?
</div>

In [ ]:
# you'll need to import the RF classifier model
from sklearn.ensemble import RandomForestClassifier

# Want to learn more?

## Continued learning with the above example
- What about other classification models?
- Try to tune the hyperparameters of the Random Forest model to find the best one.  Remember to use validation data for this.  To limit your search, try focusing on `n_estimators` and `max_depth`.
- Do the above with a form of [cross-validation](http://scikit-learn.org/0.16/modules/generated/sklearn.cross_validation.KFold.html#sklearn.cross_validation.KFold).  Don't forget to hold out test data!

## Longer term - course and book work
- A descent starting point to get an understanding of the ML landscape is to take [Andrew Ng's ML course on Coursera](https://www.coursera.org/learn/machine-learning).
- A good book which give a good sense of ML models but not in full gory detail is the [Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/).
- More in-depth reading can be found in [The Elements of Statistical Learning](https://www.amazon.com/Elements-Statistical-Learning-Prediction-Statistics/dp/0387848576) and [Machine Learning: A Probabilistic Perspective](https://www.amazon.com/Machine-Learning-Probabilistic-Perspective-Computation-ebook/dp/B00AF1AYTQ).

## Longer term - ML projects!
- One of the best ways to learn ML is to just start using it!  Pick a project to work on where you can leverage ML.  Stuck on idea's [Insight's blog](https://blog.insightdatascience.com) is full of them.
- [Kaggle is ok](https://www.kaggle.com) there is a lot of useful information on competition forums and you can learn a lot doing a project.  Be warned - 1) kaggle competitions tend to collapse down to just a handful of ML approaches and 2) kaggle is pretty focused on getting that 0.0001% improvement, which is often NOT what real world problems are calling for.